In [1]:
from selenium import webdriver
import re
import sys
sys.path.append('..')
import constants.constants as const
import constants.file_handler_constants as fh
from constants.restaurant_constants import *

from packages.restaurant.Restaurant import *
from packages.file_handler_package.file_handler import *

import os
import glob
import time
import pandas as pd

from selenium.webdriver import Remote, ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
from selenium.webdriver.common.action_chains import ActionChains
from seleniumwire import webdriver
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.edge.options import Options
# from selenium.webdriver.chrome.options import Options


from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\user\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\user\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\user\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "c:\Users\user\anaconda3\Lib\site-packages

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\user\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\user\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\user\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "c:\Users\user\anaconda3\Lib\site-packages

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\user\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\user\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\user\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "c:\Users\user\anaconda3\Lib\site-packages

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\user\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\user\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\user\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "c:\Users\user\anaconda3\Lib\site-packages

AttributeError: _ARRAY_API not found

In [ ]:
# scrape lat/long, openingHours, types, facilities (there are in adjust page of current restaurant: 'https://th.tripadvisor.com/ImproveListing-d1792735.html')
def scrape_adjust_page(restaurant_page_driver: webdriver, link_to_adjust_page: str) -> tuple[float, float, dict, list[str], list[str]]:
    lat = 0
    long = 0
    openingHours = {}
    types = []
    facilities = []
    
    # create new webdriver to continue scrape lat/long, openingHours in adjust restaurant page
    cnt_retry = 0
    
    while(True):
        # if(cnt_retry == 10):
        #     print("max retry for scrape single restaurant ...")
        #     break

        # formulate the proxy url with authentication
        proxy_url = f"http://{os.environ['proxy_username']}:{os.environ['proxy_password']}@{os.environ['proxy_address']}:{os.environ['proxy_port']}"
        
        # set selenium-wire options to use the proxy
        seleniumwire_options = {
            "proxy": {
                "http": proxy_url,
                "https": proxy_url
            },
        }

        # set Chrome options to run in headless mode
        options = webdriver.ChromeOptions()
        options.add_argument("start-maximized")
        options.add_argument("--lang=th-TH")
        # options.add_argument("--headless=new")
        options.add_experimental_option(
            "prefs", 
            {
                "profile.managed_default_content_settings.images": 2, # Disable image
                # "profile.default_content_setting_values.cookies": 2,  # Block all cookies
                "profile.default_content_settings.popups": 0,         # Disable popups
                # "profile.managed_default_content_settings.cookies": 2  # Disable third-party cookies
            }
        )

        # initialize the Chrome driver with service, selenium-wire options, and chrome options
        adjust_page_driver = webdriver.Chrome(
            service=Service(ChromeDriverManager().install()),
            seleniumwire_options=seleniumwire_options,
            options=options
        )
        
        # retry in case of web restrictions, some elements not loaded
        try:
            print("scrape data in adjust restaurant page...")
            print("for link : ", link_to_adjust_page)
            adjust_page_driver.get(link_to_adjust_page)

            print("debug option of adjust page: ")
            WebDriverWait(adjust_page_driver, 1).until(EC.visibility_of_element_located((By.CLASS_NAME, 'DiHOR')))

            # find dropdown --> click display data below --> cick display lat/long input form
            possible_target_btn = adjust_page_driver.find_elements(By.CLASS_NAME, 'DiHOR')
            for cur_dropdown_btn in possible_target_btn:
                cur_dropdown_text = cur_dropdown_btn.text
                if("แนะนำการแก้ไขข้อมูลของสถานที่นี้" in cur_dropdown_text):
                    print("found target dropdown btn ...")
                    cur_dropdown_btn.click()
                    WebDriverWait(adjust_page_driver, 1).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="lithium-root"]/main/div[2]/div[3]/div[1]/div/div[2]/div[2]/div/div/div[2]/button')))
                    # find button click to display lat/long input form
                    display_lat_long_btn = adjust_page_driver.find_element(By.XPATH, '//*[@id="lithium-root"]/main/div[2]/div[3]/div[1]/div/div[2]/div[2]/div/div/div[2]/button')
                    display_lat_long_btn.click()

        except Exception as e:
            cnt_retry += 1
            adjust_page_driver.quit()
            print("retry adjust page...")
            continue


        # scroll and wait for some msec
        adjust_page_driver.execute_script('window.scrollBy(0, document.body.scrollHeight)')

        # find lat/long
        try:
            WebDriverWait(adjust_page_driver, 1).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="lithium-root"]/main/div[2]/div[3]/div[1]/div/div[2]/div[2]/div/div/div[2]/div/div/div[3]/div[1]/div/div[2]/div/div/div/span')))
            WebDriverWait(adjust_page_driver, 1).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="lithium-root"]/main/div[2]/div[3]/div[1]/div/div[2]/div[2]/div/div/div[2]/div/div/div[3]/div[2]/div/div[2]/div/div/div/span')))
    
            lat_input_container = adjust_page_driver.find_element(By.XPATH, '//*[@id="lithium-root"]/main/div[2]/div[3]/div[1]/div/div[2]/div[2]/div/div/div[2]/div/div/div[3]/div[1]/div/div[2]/div/div/div/span')
            lat_input_element = lat_input_container.find_element(By.TAG_NAME, 'input')
            lat = float(lat_input_element.get_attribute('value'))

            long_input_container = adjust_page_driver.find_element(By.XPATH, '//*[@id="lithium-root"]/main/div[2]/div[3]/div[1]/div/div[2]/div[2]/div/div/div[2]/div/div/div[3]/div[2]/div/div[2]/div/div/div/span')
            long_input_element = long_input_container.find_element(By.TAG_NAME, 'input')
            long = float(long_input_element.get_attribute('value'))

        except Exception as e:
            print("can't find lat/long")
        
        print("lat : ", lat)
        print("long : ", long)

        # **if can't find lat/long --> don't scrape this attaction
        if(lat == 0 and long == 0):
            print("in scrape_location_latlong_openingHours --> can't find lat/long --> 0, 0")
            return lat, long, openingHours.copy(), types.copy(), facilities.copy()

        # find type
        try:     
            # find restaurant types container
            WebDriverWait(adjust_page_driver, 1).until(EC.visibility_of_element_located((By.CLASS_NAME, 'IBVyx')))

            possible_type_container = adjust_page_driver.find_elements(By.CLASS_NAME, 'IBVyx')
            type_container = None
            for cur_element in possible_type_container[::-1]:
                cur_text = cur_element.text
                if("หมวดหมู่อาหาร" in cur_text):
                    type_container = cur_element
                    WebDriverWait(adjust_page_driver, 1).until(EC.visibility_of_element_located((By.CLASS_NAME, 'RCAPL')))
                    all_type_element = type_container.find_elements(By.CLASS_NAME, 'RCAPL')
                    for cur_type in all_type_element:
                        cur_text = cur_type.text
                        types.append(cur_text)

        except Exception as e:
            print("can't find types")

        print("types --> ", types)


        # find facilities
        try:
            # find restaurant facilities container
            print("debug faci 1")
            # AoddJ
            WebDriverWait(adjust_page_driver, 1).until(EC.visibility_of_element_located((By.CLASS_NAME, 'mowmC')))
            print("debug faci 2")
            possible_facility_container = adjust_page_driver.find_elements(By.CLASS_NAME, 'mowmC')
            print("debug faci 3")
            facility_container = None
            for cur_element in possible_facility_container[::-1]:
                cur_text = cur_element.text
                print("check facility cur_text --> ", cur_text)

                if("สิ่งอำนวยความสะดวก" in cur_text):
                    facility_container = cur_element
                    print("debug faci 4")
                    WebDriverWait(adjust_page_driver, 2).until(EC.visibility_of_element_located((By.CLASS_NAME, 'PMWyE')))
                    time.sleep(15)
                    print("debug faci 5")
                    checkbox_containers = facility_container.find_elements(By.CLASS_NAME, 'PMWyE')
                    # checkbox_containers = adjust_page_driver.find_elements(By.CLASS_NAME, 'PMWyE')
                    # checkbox_containers = facility_container.find_elements(By.CLASS_NAME, 'opEmP')
                    print("debug faci 6")
                    print("check len --> ", len(checkbox_containers))
                    for Idx in range(len(checkbox_containers)):
                        print("debug faci 7")
                        cur_checkbox = checkbox_containers[Idx].find_element(By.TAG_NAME, 'span')
                        print("debug faci 8")
                        is_check = True if cur_checkbox.get_attribute('class') != 'U' else False
                        if(is_check):
                            print("debug faci 10")
                            facilities.append(ALL_RESTAURANTS_TRIPADVISOR_FACILITIES[Idx])

                    break

        except Exception as e:
            print("can't find facilities")

        print("facilities --> ", facilities)


        # find openingHours
        try:
            WebDriverWait(adjust_page_driver, 1).until(EC.visibility_of_element_located((By.CLASS_NAME, 'dNAjp')))
            all_openingHours_container = adjust_page_driver.find_elements(By.CLASS_NAME, 'dNAjp')
            for cur_openingHours_container in all_openingHours_container:
                cur_day_element = cur_openingHours_container.find_element(By.CLASS_NAME, 'ngXxk')
                cur_day_text = cur_day_element.text.replace(":", "")

                cur_time_element = cur_openingHours_container.find_element(By.CLASS_NAME, 'KxBGd')
                cur_time_text = cur_time_element.text

                openingHours[cur_day_text] = cur_time_text

        except Exception as e:
            print("can't find openingHours ...")

        print("openingHours : ", openingHours)

        time.sleep(50000)

        adjust_page_driver.quit()
        break

    return lat, long, openingHours.copy(), types.copy(), facilities.copy()

In [ ]:

restaurant = Restaurant()
cnt_retry = 0

while(True):
    # if(cnt_retry == 10):
    #     print("max retry for scrape single restaurant ...")
    #     break

    # formulate the proxy url with authentication
    proxy_url = f"http://{os.environ['proxy_username']}:{os.environ['proxy_password']}@{os.environ['proxy_address']}:{os.environ['proxy_port']}"
    
    # set selenium-wire options to use the proxy
    seleniumwire_options = {
        "proxy": {
            "http": proxy_url,
            "https": proxy_url
        },
    }

    # set web browser options to run
    options = webdriver.ChromeOptions()
    options.add_argument("start-maximized")
    options.add_argument("--lang=th-TH")
    options.add_experimental_option(
        "prefs", 
        {
            "profile.managed_default_content_settings.images": 2, # Disable image
            # "profile.default_content_setting_values.cookies": 2,  # Block all cookies
            "profile.default_content_settings.popups": 0,         # Disable popups
            # "profile.managed_default_content_settings.cookies": 2  # Disable third-party cookies
        }
    )

    # initialize the web driver with service, selenium-wire options, and web browser options
    # update: just change browser to chrome driver since priceRange can not be found in Edge
    restaurant_page_driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        seleniumwire_options=seleniumwire_options,
        options=options
    )
    link_to_restaurant = 'https://th.tripadvisor.com/Restaurant_Review-g1215781-d13349831-Reviews-Tantitium_Restaurant-Phuket_Town_Phuket.html'
    # convert restaurant url to adjust page url
    # for example: from 'https://th.tripadvisor.com/Restaurant_Review-g1210687-d1792735-Reviews-Kwong_Shop_Seafood-Kata_Beach_Karon_Phuket.html' to 'https://th.tripadvisor.com/ImproveListing-d586602.html'
    link_to_adjust_page = 'https://th.tripadvisor.com/ImproveListing-%s.html' % (link_to_restaurant.split('-')[2])

    # ** find lat/long, location data and openingHours (there are in another page of current restaurant)
    # ** if this restaurant not have lat/long
    # ** don't continue to scrape
    lat, long, openingHours, types, facilities = scrape_adjust_page(
        restaurant_page_driver = restaurant_page_driver,
        link_to_adjust_page = link_to_adjust_page
    )
    
    restaurant_page_driver.quit()
    break


scrape data in adjust restaurant page...
for link :  https://th.tripadvisor.com/ImproveListing-d13349831.html
debug option of adjust page: 
found target dropdown btn ...
lat :  7.88663
long :  98.38574
types -->  ['ฟิวชั่น', 'ไทย']
debug faci 1
debug faci 2
debug faci 3
check facility cur_text -->  ราคา: ร้านอาหารแห่งนี้ราคาแพงเท่าใด
(ไม่บังคับ)
check facility cur_text -->  ร้านอาหารแห่งนี้มีคุณลักษณะและบริการและสิ่งอำนวยความสะดวกอะไรบ้าง
(ไม่บังคับ)
debug faci 4
debug faci 5
debug faci 6
check len -->  39
debug faci 7
debug faci 8
debug faci 9
debug faci 7
debug faci 8
debug faci 9
debug faci 7
debug faci 8
debug faci 9
debug faci 7
debug faci 8
debug faci 9
debug faci 10
debug faci 7
debug faci 8
debug faci 9
debug faci 10
debug faci 7
debug faci 8
debug faci 9
debug faci 7
debug faci 8
debug faci 9
debug faci 7
debug faci 8
debug faci 9
debug faci 7
debug faci 8
debug faci 9
debug faci 10
debug faci 7
debug faci 8
debug faci 9
debug faci 10
debug faci 7
debug faci 8
debug faci 9
deb

In [3]:
def extract_type_from_url(url: str) -> str:
    all_params_containers_str = url.split('/')[-1].split('&')
    
    if(len(all_params_containers_str) == 1):
        return ''
    
    for cur_container_str in all_params_containers_str:
        params, val = cur_container_str.split('=')
        if(params == 'type'):
            return val

url = "https://th.tripadvisor.com/Restaurant_Review-g2400063-d12328548-Reviews-Minami_Sushi_Phuket-Bang_Tao_Beach_Choeng_Thale_Thalang_District_Phuket.html#/media/12328548/?albumid=101&type=ALL_INCLUDING_RESTRICTED&category=101"
type_value = extract_type_from_url(url=url)

print(type_value)

ALL_INCLUDING_RESTRICTED


In [3]:
all_url_by_page = [i for i in range(1,31)]
print(all_url_by_page[0:len(all_url_by_page)//2])
print(all_url_by_page[len(all_url_by_page)//2:])

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
[16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]


In [4]:
for item in all_url_by_page[0:len(all_url_by_page)//2]:
    print(item)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [5]:
for item in all_url_by_page[len(all_url_by_page)//2:]:
    print(item)

16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
